Loading Dataset and Preprocessing

In [1]:
import pandas as pd

data = pd.read_csv('Online Retail.csv')

data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [2]:
data.dropna(inplace = True)
data.drop_duplicates(subset = ['InvoiceNo', 'InvoiceDate'], keep = 'first', inplace = True)

In [3]:
q1 = data['Quantity'].quantile(0.25)
q3 = data['Quantity'].quantile(0.75)
iqr = q3 - q1
data = data[(data['Quantity'] >= q1 - 1.5 * iqr) & (data['Quantity'] <= q3 + 1.5 * iqr)]
data['Quantity'] = data['Quantity'].abs()
data['UnitPrice'] = data['UnitPrice'].abs()

In [4]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,20623.0,7.201183,6.990775,1.0,2.00,4.00,12.00,28.0
UnitPrice,20623.0,29.297642,485.932774,0.0,1.45,2.75,5.35,38970.0
CustomerID,20623.0,15236.928672,1654.673160,12347.0,13900.50,15287.00,16626.00,18287.0
